<a href="https://colab.research.google.com/github/geraldma1998/client-exit-predictor/blob/master/Client_exit_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Classification 

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
import io

In [0]:
uploaded = files.upload()
dataset = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))

Saving Churn_Modelling.csv to Churn_Modelling (2).csv


In [0]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [0]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,Normalizer
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

# The last arg ([1]) is the list of columns you want to transform in this step
ct = ColumnTransformer(
        # the column numbers to be transformed 
        [('one_hot_encoder', OneHotEncoder(drop='first', categories='auto'),  [1])],    
        remainder='passthrough'  # leave the rest of the columns intact
)
X = ct.fit_transform(X)

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
#Keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential() #ANN model

In [0]:
#Layers
classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_shape = (11,)))
classifier.add(Dropout(p = 0.1))

classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))

classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))

In [0]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [0]:
classifier.fit(X_train,y_train, epochs=100, batch_size=10)

In [0]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [0]:
cm

array([[1545,   50],
       [ 263,  142]])

In [0]:
#Prediction test
new_prediction = classifier.predict(sc.transform(np.array([[0.0,0,600,1,40,3,60000,2,1,1,50000]])))
new_prediction_result = (new_prediction > 0.5)
print(new_prediction,new_prediction_result)

In [0]:
# K fold validation
#Keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
  classifier = Sequential()
  #Layers
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_shape = (11,)))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "nadam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier, epochs=500, batch_size=25)

In [0]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean,variance)

0.8453749964013696 0.01938950826233554


In [0]:
#Tuning ANN parameters for get the best hyperparameters combination

#Keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
  classifier = Sequential()
  #Layers
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_shape = (11,)))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {"batch_size": [25], # 30 40
              "epochs": [100], # 50, 200
              "optimizer": ["nadam"]} #"adam" "adagrad"

grid_search = GridSearchCV(estimator = classifier,
                          param_grid = parameters,
                          scoring = "accuracy",
                          cv = 10)

grid_search = grid_search.fit(X_train,y_train)


Epoch 1/100
7200/7200 [==============================] - 8s 1ms/step - loss: 0.4983 - acc: 0.7949
Epoch 2/100
7200/7200 [==============================] - 1s 196us/step - loss: 0.4282 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 198us/step - loss: 0.4220 - acc: 0.8139
Epoch 4/100
7200/7200 [==============================] - 1s 200us/step - loss: 0.4152 - acc: 0.8272
Epoch 5/100
7200/7200 [==============================] - 1s 196us/step - loss: 0.4111 - acc: 0.8301
Epoch 6/100
7200/7200 [==============================] - 1s 198us/step - loss: 0.4077 - acc: 0.8325
Epoch 7/100
7200/7200 [==============================] - 1s 198us/step - loss: 0.4050 - acc: 0.8329
Epoch 8/100
7200/7200 [==============================] - 1s 194us/step - loss: 0.4034 - acc: 0.8325
Epoch 9/100
7200/7200 [==============================] - 1s 194us/step - loss: 0.4022 - acc: 0.8322
Epoch 10/100
7200/7200 [==============================] - 1s 196us/step - loss: 0.4009 - acc: 0.8350
E

In [0]:
best_parameter = grid_search.best_params_

best_accuracy = grid_search.best_score_

print(best_parameter,best_accuracy)

{'batch_size': 25, 'epochs': 100, 'optimizer': 'nadam'} 0.842875
